In [6]:
# Check on gpu
!nvidia-smi

Thu Apr 17 15:48:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:3B:00.0 Off |                    0 |
| N/A   29C    P0             36W /  250W |    8977MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from tifffile import imread, imwrite

import os
from glob import glob
from tqdm import tqdm

In [8]:
# Initialize Cellpose model
import torch
if torch.cuda.is_available():
    gpu = True
else:
    gpu = False
model = models.Cellpose(gpu=gpu, model_type='cyto3')

In [9]:
# Segmentation
# Set the master folder path here
master_folder = '/home/vil945/live_cell_imaging/2025-03-26_live_cell_imaging'

# Recursively find folders with '_YFP' in their names
target_folders = []
for root, dirs, files in os.walk(master_folder):
    for dir_name in dirs:
        if '_YFP' in dir_name:
            target_folders.append(os.path.join(root, dir_name))

# Process each matching folder
for folder in tqdm(target_folders, desc="Processing target folders"):
    input_files = glob(os.path.join(folder, '*.tif'))
    if not input_files:
        continue  # Skip empty folders

    # Create output folder inside the current one
    folder_name = os.path.basename(folder)
    output_folder = os.path.join(folder, f'{folder_name}_segmented')
    os.makedirs(output_folder, exist_ok=True)

    for filepath in tqdm(input_files, desc=f"Segmenting in {folder_name}", leave=False):
        # Read multi-frame TIFF
        img_stack = imread(filepath)
        n_frames = img_stack.shape[0]

        label_stack = []

        # Segment each frame
        for i in range(n_frames):
            current_label = model.eval(img_stack[i], diameter=None, channels=[0,0])[0]
            label_stack.append(current_label)

        # Save labeled multi-frame image
        label_stack = np.stack(label_stack).astype(np.uint16)
        filename = os.path.basename(filepath)
        save_path = os.path.join(output_folder, f'segmented_{filename}')
        imwrite(save_path, label_stack)

print("All segmentations complete.")

Processing target folders: 100%|██████████| 1/1 [34:56<00:00, 2096.83s/it]  

All segmentations complete.
